# 🌸 Flowers Classification with Pre-trained CNNs (Colab Project)
---
This notebook trains image classifiers using **VGG16** and **MobileNetV2** on the Flowers Recognition dataset.
It includes dataset download, preprocessing, training, evaluation, and saving models for deployment with Streamlit.

In [ ]:
# Step 1: Mount Google Drive (optional, for saving models)
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Step 2: Install & Set up Kaggle API
!pip install -q kaggle

# Upload kaggle.json (download from Kaggle > Account > Create API Token)
from google.colab import files
files.upload()  # Upload kaggle.json file here

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
# Step 3: Download Flowers Recognition dataset
!kaggle datasets download -d alxmamaev/flowers-recognition

import zipfile
with zipfile.ZipFile("flowers-recognition.zip", 'r') as zip_ref:
    zip_ref.extractall("flowers_dataset")

In [ ]:
# Step 4: Prepare Data Generators
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
    'flowers_dataset/flowers',
    target_size=(150, 150),
    batch_size=32,
    subset='training')

val_generator = train_datagen.flow_from_directory(
    'flowers_dataset/flowers',
    target_size=(150, 150),
    batch_size=32,
    subset='validation')

In [ ]:
# Step 5: Build Model using VGG16
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout

base_model = VGG16(weights='imagenet', include_top=False, input_shape=(150,150,3))
base_model.trainable = False

model_vgg = Sequential([
    base_model,
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(5, activation='softmax')
])

model_vgg.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
history_vgg = model_vgg.fit(train_generator, validation_data=val_generator, epochs=5)

In [ ]:
# Step 6: Build Model using MobileNetV2
from tensorflow.keras.applications import MobileNetV2

base_model2 = MobileNetV2(weights='imagenet', include_top=False, input_shape=(150,150,3))
base_model2.trainable = False

model_mobilenet = Sequential([
    base_model2,
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(5, activation='softmax')
])

model_mobilenet.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
history_mobilenet = model_mobilenet.fit(train_generator, validation_data=val_generator, epochs=5)

In [ ]:
# Step 7: Evaluate Models
import matplotlib.pyplot as plt

plt.plot(history_vgg.history['accuracy'], label='VGG16 Train')
plt.plot(history_vgg.history['val_accuracy'], label='VGG16 Val')
plt.plot(history_mobilenet.history['accuracy'], label='MobileNet Train')
plt.plot(history_mobilenet.history['val_accuracy'], label='MobileNet Val')
plt.legend()
plt.show()

In [ ]:
# Step 8: Save Best Model
model_vgg.save('/content/drive/MyDrive/flowers_vgg16.h5')
model_mobilenet.save('/content/drive/MyDrive/flowers_mobilenet.h5')

---
## 🚀 Streamlit Deployment (app.py)
```python
import streamlit as st
import tensorflow as tf
from tensorflow.keras.preprocessing import image
import numpy as np

model = tf.keras.models.load_model('flowers_vgg16.h5')
class_names = ['daisy','dandelion','rose','sunflower','tulip']

st.title('🌸 Flower Classifier')
file = st.file_uploader('Upload an image', type=['jpg','png','jpeg'])
if file:
    img = image.load_img(file, target_size=(150,150))
    img_array = image.img_to_array(img)/255.0
    img_array = np.expand_dims(img_array, axis=0)
    pred = np.argmax(model.predict(img_array))
    st.write(f'Prediction: {class_names[pred]}')
```
---